# Parte técnica - Projeto Redes Sociais

## Feito por: Gabriel Kabbani, Luis Filipe Loureiro e Murilo Menezes

In [1]:
from graph_tool import centrality, clustering, Graph
import pandas as pd
from collections import Counter
import seaborn as sns
sns.set()
import netpixi
from netpixi.data.gt import *
import csv

In [2]:


lista_rows=[]
anos=['2016','2017','2018','2019','2020']

df = pd.read_csv('camara_municipal_sp_sessoes_12_20.csv')

indexes_to_drop = []


for i, r in df.iterrows():
    if r['data'][6:10] not in anos or 'PL' not in r['materia']:
        indexes_to_drop.append(i)

clean_df = df.drop(df.index[indexes_to_drop]).dropna()



In [3]:
clean_df

,vereador,partido,id_parlamentar,voto,id_votacao,tipo_votacao,materia,ementa,resultado,presentes,votacao_sim,votacao_nao,votacao_abstencao,votacao_branco,notas,sessao,data
35049,ABOU ANNI,PV,1297.0,Não,2105,Nominal,Sub 1 ao PL 272 /2015 DO EXECUTIVO,"DISCIPLINA O PARCELAMENTO, O USO E A OCUPAÇÃO...",Aprovado,55.0,45.0,8.0,0.0,0.0,APROVADO EM SEGUNDA DISCUSSÃO E VOTAÇÃO NA FOR...,308ª Sessão Extraordinária,25/02/2016
35050,ADILSON AMADEU,PTB,1296.0,Sim,2105,Nominal,Sub 1 ao PL 272 /2015 DO EXECUTIVO,"DISCIPLINA O PARCELAMENTO, O USO E A OCUPAÇÃO...",Aprovado,55.0,45.0,8.0,0.0,0.0,APROVADO EM SEGUNDA DISCUSSÃO E VOTAÇÃO NA FOR...,308ª Sessão Extraordinária,25/02/2016
35051,ALESSANDRO GUEDES,PT,2000.0,Sim,2105,Nominal,Sub 1 ao PL 272 /2015 DO EXECUTIVO,"DISCIPLINA O PARCELAMENTO, O USO E A OCUPAÇÃO...",Aprovado,55.0,45.0,8.0,0.0,0.0,APROVADO EM SEGUNDA DISCUSSÃO E VOTAÇÃO NA FOR...,308ª Sessão Extraordinária,25/02/2016
35052,ALFREDINHO,PT,1447.0,Sim,2105,Nominal,Sub 1 ao PL 272 /2015 DO EXECUTIVO,"DISCIPLINA O PARCELAMENTO, O USO E A OCUPAÇÃO...",Aprovado,55.0,45.0,8.0,0.0,0.0,APROVADO EM SEGUNDA DISCUSSÃO E VOTAÇÃO NA FOR...,308ª Sessão Extraordinária,25/02/2016
35053,ANDREA MATARAZZO,PSD,1994.0,Não,2105,Nominal,Sub 1 ao PL 272 /2015 DO EXECUTIVO,"DISCIPLINA O PARCELAMENTO, O USO E A OCUPAÇÃO...",Aprovado,55.0,45.0,8.0,0.0,0.0,APROVADO EM SEGUNDA DISCUSSÃO E VOTAÇÃO NA FOR...,308ª Sessão Extraordinária,25/02/2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62273,SOUZA SANTOS,REPUBLICANOS,1633.0,Sim,2922,Nominal,"Sub ao PL 643/2020, da COMISSÃO DE FINANÇAS E ...",Estima a receita e fixa a despesa do Municípi...,Aprovado,55.0,37.0,12.0,0.0,0.0,FASE: 2ª DO SUBSTITUTIVO Nº 1 DA COMISSÃO DE F...,300ª Sessão Extraordinária,23/12/2020
62274,TONINHO PAIVA,PL,220.0,Sim,2922,Nominal,"Sub ao PL 643/2020, da COMISSÃO DE FINANÇAS E ...",Estima a receita e fixa a despesa do Municípi...,Aprovado,55.0,37.0,12.0,0.0,0.0,FASE: 2ª DO SUBSTITUTIVO Nº 1 DA COMISSÃO DE F...,300ª Sessão Extraordinária,23/12/2020
62275,TONINHO VESPOLI,PSOL,1999.0,Não,2922,Nominal,"Sub ao PL 643/2020, da COMISSÃO DE FINANÇAS E ...",Estima a receita e fixa a despesa do Municípi...,Aprovado,55.0,37.0,12.0,0.0,0.0,FASE: 2ª DO SUBSTITUTIVO Nº 1 DA COMISSÃO DE F...,300ª Sessão Extraordinária,23/12/2020
62276,XEXÉU TRIPOLI,PSDB,2158.0,Sim,2922,Nominal,"Sub ao PL 643/2020, da COMISSÃO DE FINANÇAS E ...",Estima a receita e fixa a despesa do Municípi...,Aprovado,55.0,37.0,12.0,0.0,0.0,FASE: 2ª DO SUBSTITUTIVO Nº 1 DA COMISSÃO DE F...,300ª Sessão Extraordinária,23/12/2020


In [4]:
clean_df['voto'].value_counts()

Sim          7181
Não          2466
Abstenção     150
Name: voto, dtype: int64

In [5]:
vereadores = []
projetos = []

for i, r in clean_df.iterrows():
    if r['vereador'] not in vereadores:
        vereadores.append(r['vereador'])
    if r['materia'] not in projetos:
        projetos.append(r['materia'])


In [6]:
g = Graph(directed=True)

g.vp['bipartite'] = g.new_vp('object')
g.vp['nome'] = g.new_vp('object')

In [7]:
for _ in vereadores:
    g.add_vertex()
    
i=0
for v in g.vertices():
    g.vp['bipartite'][v]= 1
    g.vp['nome'][v]=vereadores[i]
    i+=1
    
for _ in projetos:
    g.add_vertex()
    
    
i=0
for v in g.vertices():
    if g.vp['bipartite'][v] != 1:
        g.vp['bipartite'][v]= 0
        g.vp['nome'][v]=projetos[i]
        i+=1
        
        


In [8]:
for v in g.vertices():
    print(g.vp['bipartite'][v], g.vp['nome'][v])

1 ABOU ANNI
1 ADILSON AMADEU
1 ALESSANDRO GUEDES
1 ALFREDINHO
1 ANDREA MATARAZZO
1 ANIBAL DE FREITAS FILHO
1 ANTONIO DONATO
1 ARI FRIEDENBACH
1 ARSELINO TATTO
1 ATILIO FRANCISCO
1 AURÉLIO MIGUEL
1 AURELIO NOMURA
1 CALVO
1 CLAUDINHO DE SOUZA
1 CONTE LOPES
1 DALTON SILVANO
1 DAVID SOARES
1 EDIR SALES
1 EDUARDO TUMA
1 ELISEU GABRIEL
1 FRANCISCO CHAGAS
1 GEORGE HATO
1 GILSON BARRETO
1 JAIR TATTO
1 JAMIL MURAD
1 JONAS CAMISA NOVA
1 JOSÉ POLICE NETO
1 JULIANA CARDOSO
1 MÁRIO COVAS NETO
1 MARQUITO
1 MILTON LEITE
1 NATALINI
1 NELO RODOLFO
1 NOEMI NONATO
1 OTA
1 PATRÍCIA BEZERRA
1 PAULO FIORILO
1 PAULO FRANGE
1 PR. EDEMILSON CHAVES
1 QUITO FORMIGA
1 REIS
1 RICARDO NUNES
1 RICARDO TEIXEIRA
1 RICARDO YOUNG
1 SALOMÃO PEREIRA
1 SANDRA TADEU
1 SENIVAL MOURA
1 SOUZA SANTOS
1 TONINHO PAIVA
1 TONINHO VESPOLI
1 USHITARO KAMIA
1 VALDECIR CABRABOM
1 VAVÁ
1 ADOLFO QUINTAS
1 LAÉRCIO BENKO
1 WADIH MUTRAN
1 CELSO JATENE
1 NABIL BONDUKI
1 JOSELITO
1 JEAN MADEIRA
1 RODOLFO DESPACHANTE
1 ADRIANA RAMALHO
1 ALINE 

In [9]:
save_gt(g, 'exemplo.net.gz')
r = netpixi.render('exemplo.net.gz')

In [10]:
#utilizando o algoritmo SFDP, que utiliza force-directed graph drawing para aproximar os vértices conectados

g = load_gt('exemplo.net.gz')
m = draw_gt.sfdp_layout(g)
move_gt(g, m)
save_gt(g, 'exemplo.net.gz')
netpixi.render('exemplo.net.gz')

In [11]:
g.vp['color'] = g.new_vp('object')

for v in g.vertices():
    if g.vp['bipartite'][v] == 0:
        g.vp['color'][v] = 0xff0000
    else:
        g.vp['color'][v] = 0x00ff00

save_gt(g, 'exemplo.net.gz')
netpixi.render('exemplo.net.gz')

In [12]:
g.ep['voto'] = g.new_ep('object')
for v in g.vertices():
    for v2 in g.vertices():
        if g.vp['bipartite'][v] == 0 and g.vp['bipartite'][v2] == 1:
            if g.edge(v,v2)==None:
                g.add_edge(v, v2)
                row=clean_df.loc[(clean_df["vereador"] == g.vp['nome'][v2]) & (clean_df["materia"]==g.vp['nome'][v])]
                e = g.edge(v,v2)
                #print(row['voto'].tolist())
                if (row['voto'].tolist()) == ['Não']:
                    g.ep['voto'][e]=-1
                elif (row['voto'].tolist()) == ['Sim']:
                    g.ep['voto'][e]=1
                else:
                    g.ep['voto'][e]=0  

In [13]:
for i, r in clean_df.iterrows():
    print(r['vereador'])

ABOU ANNI
ADILSON AMADEU
ALESSANDRO GUEDES
ALFREDINHO
ANDREA MATARAZZO
ANIBAL DE FREITAS FILHO
ANTONIO DONATO
ARI FRIEDENBACH
ARSELINO TATTO
ATILIO FRANCISCO
AURÉLIO MIGUEL
AURELIO NOMURA
CALVO
CLAUDINHO DE SOUZA
CONTE LOPES
DALTON SILVANO
DAVID SOARES
EDIR SALES
EDUARDO TUMA
ELISEU GABRIEL
FRANCISCO CHAGAS
GEORGE HATO
GILSON BARRETO
JAIR TATTO
JAMIL MURAD
JONAS CAMISA NOVA
JOSÉ POLICE NETO
JULIANA CARDOSO
MÁRIO COVAS NETO
MARQUITO
MILTON LEITE
NATALINI
NELO RODOLFO
NOEMI NONATO
OTA
PATRÍCIA BEZERRA
PAULO FIORILO
PAULO FRANGE
PR. EDEMILSON CHAVES
QUITO FORMIGA
REIS
RICARDO NUNES
RICARDO TEIXEIRA
RICARDO YOUNG
SALOMÃO PEREIRA
SANDRA TADEU
SENIVAL MOURA
SOUZA SANTOS
TONINHO PAIVA
TONINHO VESPOLI
USHITARO KAMIA
VALDECIR CABRABOM
VAVÁ
ADILSON AMADEU
ADOLFO QUINTAS
ALESSANDRO GUEDES
ALFREDINHO
ANDREA MATARAZZO
ANIBAL DE FREITAS FILHO
ANTONIO DONATO
ARI FRIEDENBACH
ARSELINO TATTO
AURÉLIO MIGUEL
AURELIO NOMURA
CLAUDINHO DE SOUZA
CONTE LOPES
DALTON SILVANO
DAVID SOARES
EDIR SALES
EDUARDO TUMA


NABIL BONDUKI
NATALINI
NELO RODOLFO
NOEMI NONATO
OTA
PAULO FIORILO
PAULO FRANGE
PR. EDEMILSON CHAVES
REIS
RICARDO NUNES
RICARDO TEIXEIRA
SALOMÃO PEREIRA
SALOMÃO PEREIRA
SANDRA TADEU
SENIVAL MOURA
SOUZA SANTOS
TONINHO PAIVA
VAVÁ
WADIH MUTRAN
ABOU ANNI
ALFREDINHO
ANTONIO DONATO
ARSELINO TATTO
ATILIO FRANCISCO
AURELIO NOMURA
CALVO
CELSO JATENE
CLAUDINHO DE SOUZA
CONTE LOPES
DALTON SILVANO
DAVID SOARES
EDIR SALES
EDUARDO TUMA
ELISEU GABRIEL
JOSELITO
GILSON BARRETO
JAIR TATTO
JEAN MADEIRA
JONAS CAMISA NOVA
JOSÉ POLICE NETO
LAÉRCIO BENKO
MÁRIO COVAS NETO
NABIL BONDUKI
NATALINI
NELO RODOLFO
NOEMI NONATO
OTA
PAULO FIORILO
PR. EDEMILSON CHAVES
REIS
RICARDO NUNES
RICARDO TEIXEIRA
SALOMÃO PEREIRA
SANDRA TADEU
SENIVAL MOURA
TONINHO PAIVA
VAVÁ
WADIH MUTRAN
ABOU ANNI
ADOLFO QUINTAS
ALFREDINHO
ANDREA MATARAZZO
ANIBAL DE FREITAS FILHO
ANTONIO DONATO
ARI FRIEDENBACH
ARSELINO TATTO
ATILIO FRANCISCO
AURÉLIO MIGUEL
AURELIO NOMURA
CALVO
CELSO JATENE
CONTE LOPES
DALTON SILVANO
DAVID SOARES
EDUARDO TUMA
ELIS

QUITO FORMIGA
REIS
RICARDO NUNES
RICARDO TEIXEIRA
RINALDI DIGILIO
RODRIGO GOMES
RODRIGO GOULART
RUTE COSTA
SÂMIA BOMFIM
SANDRA TADEU
SENIVAL MOURA
SOUZA SANTOS
TONINHO VESPOLI
REGINALDO TRÍPOLI
ZÉ TURIN
ABOU ANNI
ADILSON AMADEU
ADRIANA RAMALHO
ALESSANDRO GUEDES
ALFREDINHO
ALINE CARDOSO
ANDRÉ SANTOS
ANTONIO DONATO
ATÍLIO FRANCISCO
AURÉLIO NOMURA
CAIO MIRANDA CARNEIRO
CAMILO CRISTÓFARO
CELSO JATENE
CLAUDIO FONSECA
CONTE LOPES
DALTON SILVANO
DAVID SOARES
EDIR SALES
EDUARDO MATARAZZO SUPLICY
EDUARDO TUMA
FABIO RIVA
FERNANDO HOLIDAY
GEORGE HATO
GILBERTO NASCIMENTO
GILSON BARRETO
ISAC FELIX
JAIR TATTO
JANAÍNA LIMA
JOÃO JORGE
JULIANA CARDOSO
MARIO COVAS NETO
MILTON FERREIRA
MILTON LEITE
OTA
JOSÉ POLICE NETO
QUITO FORMIGA
REIS
RICARDO NUNES
RICARDO TEIXEIRA
RINALDI DIGILIO
RODRIGO GOMES
RODRIGO GOULART
RUTE COSTA
SÂMIA BOMFIM
SANDRA TADEU
SENIVAL MOURA
SOUZA SANTOS
TONINHO VESPOLI
REGINALDO TRÍPOLI
ZÉ TURIN
ABOU ANNI
ADILSON AMADEU
ADRIANA RAMALHO
ALINE CARDOSO
ANDRÉ SANTOS
ATÍLIO FRANCISCO
AU

RUTE COSTA
SÂMIA BOMFIM
SANDRA TADEU
SONINHA FRANCINE
SOUZA SANTOS
TONINHO PAIVA
TONINHO VESPOLI
ZÉ TURIN
ABOU ANNI
ADILSON AMADEU
ADRIANA RAMALHO
ALESSANDRO GUEDES
ALFREDINHO
ALINE CARDOSO
ANDRÉ SANTOS
ANTONIO DONATO
ARSELINO TATTO
ATÍLIO FRANCISCO
AURÉLIO NOMURA
CAIO MIRANDA CARNEIRO
CAIO MIRANDA CARNEIRO
CAMILO CRISTÓFARO
CELSO JATENE
CONTE LOPES
DALTON SILVANO
EDIR SALES
EDUARDO MATARAZZO SUPLICY
EDUARDO TUMA
FABIO RIVA
FERNANDO HOLIDAY
GEORGE HATO
GILBERTO NASCIMENTO
GILSON BARRETO
ISAC FELIX
JANAÍNA LIMA
JOÃO JORGE
JULIANA CARDOSO
MILTON FERREIRA
MILTON LEITE
NOEMI NONATO
OTA
PATRÍCIA BEZERRA
PAULO FRANGE
JOSÉ POLICE NETO
REIS
RICARDO NUNES
RICARDO TEIXEIRA
RINALDI DIGILIO
RODRIGO GOULART
RUTE COSTA
SÂMIA BOMFIM
SANDRA TADEU
SONINHA FRANCINE
SOUZA SANTOS
TONINHO PAIVA
TONINHO VESPOLI
ZÉ TURIN
ADILSON AMADEU
ADRIANA RAMALHO
ALINE CARDOSO
ANDRÉ SANTOS
ATÍLIO FRANCISCO
AURÉLIO NOMURA
CAIO MIRANDA CARNEIRO
CAMILO CRISTÓFARO
CLAUDINHO DE SOUZA
CLAUDIO FONSECA
CONTE LOPES
DALTON SILVAN

RICARDO NUNES
RICARDO TEIXEIRA
RODRIGO GOULART
SÂMIA BOMFIM
SANDRA TADEU
SONINHA FRANCINE
SOUZA SANTOS
TONINHO PAIVA
REGINALDO TRÍPOLI
ZÉ TURIN
ADILSON AMADEU
ADRIANA RAMALHO
ALFREDINHO
ANDRÉ SANTOS
ANTONIO DONATO
ATÍLIO FRANCISCO
AURÉLIO NOMURA
CAIO MIRANDA CARNEIRO
CAMILO CRISTÓFARO
CELSO JATENE
CLAUDINHO DE SOUZA
CLAUDIO FONSECA
CONTE LOPES
DALTON SILVANO
DAVID SOARES
EDIR SALES
EDUARDO MATARAZZO SUPLICY
EDUARDO TUMA
ELISEU GABRIEL
FABIO RIVA
FERNANDO HOLIDAY
GEORGE HATO
GILBERTO NASCIMENTO
GILSON BARRETO
JOÃO JORGE
JULIANA CARDOSO
MARIO COVAS NETO
MILTON FERREIRA
MILTON LEITE
NOEMI NONATO
OTA
PATRÍCIA BEZERRA
PAULO FRANGE
JOSÉ POLICE NETO
REIS
RICARDO NUNES
RICARDO TEIXEIRA
RINALDI DIGILIO
RODRIGO GOULART
SÂMIA BOMFIM
SANDRA TADEU
SENIVAL MOURA
SONINHA FRANCINE
SOUZA SANTOS
TONINHO PAIVA
TONINHO VESPOLI
ZÉ TURIN
ADILSON AMADEU
ADRIANA RAMALHO
ALFREDINHO
ANDRÉ SANTOS
ANTONIO DONATO
ATÍLIO FRANCISCO
AURÉLIO NOMURA
CAIO MIRANDA CARNEIRO
CLAUDINHO DE SOUZA
CLAUDIO FONSECA
CONTE LOPES
D

...

ATÍLIO FRANCISCO
AURÉLIO NOMURA
BETO DO SOCIAL
CAIO MIRANDA CARNEIRO
CAMILO CRISTÓFARO
CELSO GIANNAZI
CELSO JATENE
CLAUDINHO DE SOUZA
CLAUDIO FONSECA
DALTON SILVANO
EDIR SALES
EDUARDO MATARAZZO SUPLICY
EDUARDO TUMA
ELISEU GABRIEL
FABIO RIVA
FERNANDO HOLIDAY
GEORGE HATO
GILBERTO NASCIMENTO
GILSON BARRETO
ISAC FELIX
JANAÍNA LIMA
JULIANA CARDOSO
MARIO COVAS NETO
MILTON FERREIRA
MILTON LEITE
NOEMI NONATO
OTA
PATRÍCIA BEZERRA
PAULO FRANGE
QUITO FORMIGA
REIS
RICARDO NUNES
RINALDI DIGILIO
RODRIGO GOULART
RUTE COSTA
SANDRA TADEU
SENIVAL MOURA
SONINHA FRANCINE
SOUZA SANTOS
TONINHO PAIVA
TONINHO VESPOLI
XEXÉU TRIPOLI
ZÉ TURIN
ADRIANA RAMALHO
ALESSANDRO GUEDES
ALFREDINHO
ANDRÉ SANTOS
ANTONIO DONATO
ARSELINO TATTO
ATÍLIO FRANCISCO
AURÉLIO NOMURA
CAIO MIRANDA CARNEIRO
CELSO GIANNAZI
CELSO JATENE
CLAUDINHO DE SOUZA
CLAUDIO FONSECA
DALTON SILVANO
EDIR SALES
EDUARDO MATARAZZO SUPLICY
EDUARDO TUMA
ELISEU GABRIEL
FABIO RIVA
FERNANDO HOLIDAY
GEORGE HATO
GILBERTO NASCIMENTO
GILSON BARRETO
ISAC FELIX
JAIR 

ZÉ TURIN
ADILSON AMADEU
ALFREDINHO
ANTONIO DONATO
ARSELINO TATTO
ATÍLIO FRANCISCO
CELSO GIANNAZI
CLAUDINHO DE SOUZA
CLAUDIO FONSECA
DANIEL ANNENBERG
EDUARDO MATARAZZO SUPLICY
ELISEU GABRIEL
FABIO RIVA
FERNANDO HOLIDAY
GEORGE HATO
GILBERTO NATALINI
JOÃO JORGE
JOSÉ POLICE NETO
JULIANA CARDOSO
MILTON FERREIRA
NOEMI NONATO
PATRÍCIA BEZERRA
PAULO FRANGE
QUITO FORMIGA
REIS
RICARDO NUNES
RINALDI DIGILIO
SANDRA TADEU
SONINHA FRANCINE
TONINHO PAIVA
TONINHO VESPOLI
ADILSON AMADEU
ALFREDINHO
ANDRÉ SANTOS
ANTONIO DONATO
ARSELINO TATTO
ATÍLIO FRANCISCO
AURÉLIO NOMURA
CAIO MIRANDA CARNEIRO
CAMILO CRISTÓFARO
CAMILO CRISTÓFARO
CELSO GIANNAZI
CLAUDINHO DE SOUZA
CLAUDIO FONSECA
EDIR SALES
EDUARDO MATARAZZO SUPLICY
EDUARDO TUMA
FABIO RIVA
FERNANDO HOLIDAY
GEORGE HATO
GILBERTO NASCIMENTO
GILBERTO NATALINI
GILSON BARRETO
ISAC FELIX
JANAÍNA LIMA
JOÃO JORGE
JOSÉ POLICE NETO
JULIANA CARDOSO
MARIO COVAS NETO
MILTON FERREIRA
MILTON LEITE
NOEMI NONATO
PATRÍCIA BEZERRA
PAULO FRANGE
QUITO FORMIGA
REIS
RICARDO NUNE

RICARDO TEIXEIRA
RODRIGO GOULART
RUTE COSTA
SANDRA TADEU
SENIVAL MOURA
SONINHA FRANCINE
XEXÉU TRIPOLI
ZÉ TURIN
ADRIANA RAMALHO
ALESSANDRO GUEDES
ALFREDINHO
ANDRÉ SANTOS
ARSELINO TATTO
AURÉLIO NOMURA
CAIO MIRANDA CARNEIRO
CELSO JATENE
CLAUDIO FONSECA
DALTON SILVANO
DALTON SILVANO
DANIEL ANNENBERG
EDUARDO MATARAZZO SUPLICY
EDUARDO TUMA
ELISEU GABRIEL
FABIO RIVA
GEORGE HATO
GILBERTO NASCIMENTO
GILBERTO NATALINI
GILSON BARRETO
ISAC FELIX
JAIR TATTO
JANAÍNA LIMA
JOÃO JORGE
JOSÉ POLICE NETO
JULIANA CARDOSO
MARIO COVAS NETO
MILTON FERREIRA
PATRÍCIA BEZERRA
REIS
RICARDO NUNES
RICARDO TEIXEIRA
RINALDI DIGILIO
RODRIGO GOULART
RUTE COSTA
SANDRA TADEU
SENIVAL MOURA
SONINHA FRANCINE
XEXÉU TRIPOLI
ZÉ TURIN
ADRIANA RAMALHO
ALESSANDRO GUEDES
ALFREDINHO
ANDRÉ SANTOS
ARSELINO TATTO
AURÉLIO NOMURA
CAIO MIRANDA CARNEIRO
CELSO GIANNAZI
CELSO JATENE
CLAUDINHO DE SOUZA
CLAUDIO FONSECA
DANIEL ANNENBERG
EDUARDO MATARAZZO SUPLICY
EDUARDO TUMA
FABIO RIVA
FERNANDO HOLIDAY
GEORGE HATO
GILBERTO NASCIMENTO
GILBERTO 

CLAUDIO FONSECA
DALTON SILVANO
DANIEL ANNENBERG
EDIR SALES
EDUARDO MATARAZZO SUPLICY
EDUARDO TUMA
ELISEU GABRIEL
FABIO RIVA
FERNANDO HOLIDAY
GEORGE HATO
GILBERTO NASCIMENTO
GILBERTO NATALINI
GILSON BARRETO
ISAC FELIX
JAIR TATTO
JANAÍNA LIMA
JANAÍNA LIMA
JOÃO JORGE
JULIANA CARDOSO
MARIO COVAS NETO
MILTON FERREIRA
MILTON LEITE
NOEMI NONATO
OTA
PATRÍCIA BEZERRA
PAULO FRANGE
QUITO FORMIGA
REIS
RICARDO NUNES
RINALDI DIGILIO
RUTE COSTA
SANDRA TADEU
SENIVAL MOURA
SONINHA FRANCINE
TONINHO VESPOLI
XEXÉU TRIPOLI
ZÉ TURIN
ADILSON AMADEU
ADRIANA RAMALHO
ALESSANDRO GUEDES
ALFREDINHO
ANDRÉ SANTOS
ANTONIO DONATO
ARSELINO TATTO
ATÍLIO FRANCISCO
AURÉLIO NOMURA
CAIO MIRANDA CARNEIRO
CAMILO CRISTÓFARO
CELSO GIANNAZI
CLAUDIO FONSECA
DALTON SILVANO
DANIEL ANNENBERG
EDIR SALES
EDUARDO MATARAZZO SUPLICY
EDUARDO TUMA
ELISEU GABRIEL
FABIO RIVA
FERNANDO HOLIDAY
GEORGE HATO
GILBERTO NASCIMENTO
GILSON BARRETO
ISAC FELIX
JAIR TATTO
JANAÍNA LIMA
JOÃO JORGE
JULIANA CARDOSO
MARIO COVAS NETO
MILTON FERREIRA
MILTON LEI

ADRIANA RAMALHO
ALESSANDRO GUEDES
ALFREDINHO
ANTONIO DONATO
ARSELINO TATTO
ATÍLIO FRANCISCO
CAIO MIRANDA CARNEIRO
CAMILO CRISTÓFARO
CAMILO CRISTÓFARO
CELSO GIANNAZI
CELSO JATENE
CLAUDIO FONSECA
DALTON SILVANO
EDIR SALES
EDUARDO MATARAZZO SUPLICY
EDUARDO TUMA
FABIO RIVA
FERNANDO HOLIDAY
GILBERTO NASCIMENTO
GILSON BARRETO
ISAC FELIX
JAIR TATTO
JANAÍNA LIMA
JOÃO JORGE
JOSÉ POLICE NETO
JULIANA CARDOSO
MARIO COVAS NETO
MILTON FERREIRA
NOEMI NONATO
PATRÍCIA BEZERRA
QUITO FORMIGA
REIS
RINALDI DIGILIO
RUTE COSTA
SANDRA TADEU
SONINHA FRANCINE
TONINHO VESPOLI
XEXÉU TRIPOLI
ZÉ TURIN
ADILSON AMADEU
ADRIANA RAMALHO
ALESSANDRO GUEDES
ALFREDINHO
ANTONIO DONATO
ARSELINO TATTO
ATÍLIO FRANCISCO
AURÉLIO NOMURA
CAIO MIRANDA CARNEIRO
CAIO MIRANDA CARNEIRO
CAMILO CRISTÓFARO
CELSO GIANNAZI
CELSO JATENE
CLAUDINHO DE SOUZA
CLAUDIO FONSECA
DALTON SILVANO
DANIEL ANNENBERG
EDUARDO MATARAZZO SUPLICY
ELISEU GABRIEL
FABIO RIVA
FERNANDO HOLIDAY
GILBERTO NASCIMENTO
GILBERTO NATALINI
GILSON BARRETO
ISAC FELIX
JAIR TATT

In [14]:
save_gt(g, 'exemplo.net.gz')
netpixi.render('exemplo.net.gz')

In [15]:
produto_ver_projeto = len(vereadores)*len(projetos)
print('Vereadores vezes projetos: {}'.format(produto_ver_projeto))

Vereadores vezes projetos: 20855


In [16]:
#conferindo se numero de edges ta certo
cont=0
for e in g.edges():
    cont+=1
print(cont==produto_ver_projeto)

True


In [17]:
#verificando valor das edges
for e in g.edges():
    print(g.ep['voto'][e])

-1
1
1
1
-1
1
1
1
1
1
1
-1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
-1
1
1
-1
1
1
1
-1
1
1
1
1
1
1
1
-1
1
1
1
1
1
-1
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
-1
-1
1
-1
1
1
-1
0
1
1
0
1
1
-1
1
-1
1
0
-1
0
0
-1
0
1
1
-1
1
-1
1
0
1
-1
0
0
-1
-1
-1
1
-1
1
0
1
1
-1
-1
0
-1
0
-1
0
-1
-1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
0
1
1
1
1
1
1
-1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
0
1
1
0
1
1
0
0
1
1
1
1
1
1
1
1
0
1
1
0
1
0
1
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
-1
-1
-1
1
-1
0
-1
-1
1
-1
0
-1
1
-1
0
-1
-1
-1
0
-1
-1
-1
-1
-1
-1
1
-1
0
-1
-1
0
-1
-1
0
0
-1
-1
-1
-1
-1
0
-1
1
-1
-1
-1
0
-1
0
-1
-1
-1
-1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
-1
1
1
1
1
1
1
-1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
-1
1
1
-1
0
1
1
-1
1
1
1
1
1
1
1
-1
1
1
1
1
1
-1
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


-1
-1
-1
-1
-1
-1
0
-1
-1
-1
-1
-1
-1
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
0
0
1
0
0
0
0
0
0
1
1
1
0
1
-1
0
0
1
0
0
0
0
1
0
0
0
1
0
0
1
1
0
0
0
0
1
1
1
1
0
0
1
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
0
1
1
1
1
0
1
1
1
1
1
1
0
1
1
1
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
1
1
-1
-1
0
0
-1
0
0
0
0
0
0
1
1
1
0
1
-1
0
0
1
1
0
0
0
1
0
0
0
1
0
0
1
1
0
0
1
0
1
0
0
1
0
0
1
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
0
1
-1
1
1
0
1
0
1
1
1
1
-1
1
1
1
-1
1
1
-1
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
0
0
1
0
0
0
0
0
0
1
1
1
0
1
1
0
0
1
1
0
0
0
1
0
0
0
1
0
0
1
1
0
0
1
0
1
1
0
1
0
0
1
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
0
1
0
1
1
-1
1
0
1
1
1
1
0
1
1
1
-1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
-1
0
0
0
0
0
0
0
0
0
0
0
0
-1
-1
-1
0
0
-1
0
0
-1
-1
0
0
0
-1
1
0
0
-1
0
0
-1
-1
0
0
-1
0
-1
0
-1
0
0
0
-1
0
-1
-1
0
0
0
0
0
0
0
0
0
0
0
0
-1
-1
-1
-1
0
0
-1
1
-1
-1
-1
-1
0
-1
-1
0
0
0
-1
-1
-1
-1
0
-1
0
-1
0
0
0
0
0
0
0
0
0
0
1
1
-1
-1
0
0
-1
0
0
0
0
0
0
1
1
1
0
1
1
0
0
1
1
-1
0
0
1
0
0
0
1
0
0
1
1
0
0
1
0
1
-1
1
1
0
0
1
0
0
1
-1
0
0
0
0
0
0

1
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
1
0
0
0
0
0
0
0
1
0
1
1
1
1
0
0
1
1
0
0
1
0
0
0
1
0
0
1
1
1
0
0
0
0
1
1
0
0
0
1
1
0
0
1
0
0
0
0
0
0
1
0
0
0
0
1
0
1
1
1
0
1
0
1
1
1
1
1
1
1
0
1
1
1
1
0
1
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
1
0
0
0
0
0
0
0
1
0
1
1
1
1
0
1
1
1
0
0
1
0
0
0
1
0
0
1
1
0
0
0
0
0
1
1
0
0
0
1
1
0
0
1
0
0
0
0
0
0
1
0
0
0
0
1
0
1
1
0
0
1
0
1
1
1
1
1
1
1
0
1
0
1
1
0
1
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
1
0
0
0
0
0
0
0
1
0
1
1
1
1
0
1
1
1
0
0
0
0
0
0
1
0
0
1
1
0
0
0
0
0
1
1
0
0
0
1
1
0
0
0
0
0
0
0
0
0
1
0
0
0
0
1
0
1
1
1
0
1
0
1
1
1
1
1
1
1
0
1
0
1
1
0
1
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
1
1
1
0
0
1
0
1
0
0
0
0
0
1
0
1
0
1
1
0
1
0
1
0
0
0
0
0
0
1
0
0
1
1
1
0
0
0
0
1
1
1
0
0
0
1
1
1
-1
0
0
0
0
0
0
1
0
0
0
0
1
0
1
1
0
1
1
1
-1
1
-1
1
1
1
0
0
1
-1
1
1
1
1
1
0
0
1
1
0
0
0
0
0
0
0
0
0
0
1
1
1
0
0
1
0
1
0
0
0
0
0
1
1
1
1
1
1
0
1
1
1
0
0
1
0
0
0
1
0
0
1
1
0
0
1
0
0
1
1
1
0
0
0
1
1
1
1
0
0
0
0
0
0
1
0
0
0
0
1
0
1
1
0
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
0
0
1
1
0
0
0
0
0
0
0
0


0
0
0
0
1
0
1
0
1
1
0
0
1
1
0
0
0
-1
0
0
0
0
0
0
1
1
1
0
1
0
1
0
1
1
0
0
1
0
1
1
0
0
0
0
0
0
0
1
0
0
0
0
1
0
1
1
-1
0
-1
0
0
1
-1
0
0
1
1
0
1
0
0
1
1
1
1
0
0
0
-1
0
0
0
0
0
1
0
1
0
0
0
-1
-1
0
0
-1
0
0
0
0
0
0
1
0
1
0
1
1
1
0
1
1
0
0
0
0
-1
0
0
0
0
0
1
1
1
0
1
0
1
-1
0
1
0
0
1
0
1
1
-1
0
0
0
0
0
0
1
0
0
0
0
1
0
0
-1
1
0
-1
-1
-1
1
-1
0
-1
1
1
0
1
0
0
1
1
1
1
0
0
-1
-1
0
0
0
0
0
1
0
1
0
0
0
0
1
0
0
1
0
1
0
0
0
0
1
0
1
0
1
1
1
0
1
1
1
0
0
1
0
0
0
1
0
0
1
1
1
0
1
0
1
1
1
0
0
0
1
1
1
1
0
0
0
0
0
0
0
1
0
0
0
0
1
0
0
1
1
1
1
1
1
1
1
0
1
1
1
0
1
0
0
1
1
1
1
1
0
1
0
0
0
0
0
0
1
0
1
0
0
0
0
-1
0
0
-1
0
0
0
0
0
0
1
0
1
0
1
1
1
0
1
1
0
0
0
0
-1
0
0
1
0
0
1
1
1
0
1
0
1
-1
1
0
0
0
1
-1
1
1
-1
0
0
0
0
0
0
1
0
0
0
0
1
0
1
1
1
1
1
-1
1
1
1
0
1
1
1
0
1
0
0
1
1
1
1
1
0
1
0
0
0
0
0
0
1
-1
1
0
0
0
-1
-1
0
0
-1
0
-1
0
0
0
0
1
0
1
0
1
1
-1
0
1
1
-1
0
1
-1
-1
0
0
1
0
0
1
1
1
0
1
0
1
-1
1
1
0
0
1
-1
1
1
-1
0
0
0
0
0
0
0
0
0
0
0
1
0
1
1
0
0
-1
-1
-1
1
-1
0
1
1
0
0
1
0
0
1
1
1
-1
1
0
-1
0
0
0
0
0
0
0
-1
1
0
0
0

0
1
0
0
0
0
1
0
0
0
0
1
0
1
0
0
1
1
0
0
1
0
0
0
0
0
0
1
0
0
0
0
1
0
1
0
1
0
1
1
-1
1
1
1
1
1
1
0
1
0
0
0
0
1
1
1
0
1
1
0
0
0
0
1
0
1
0
0
0
1
1
1
0
0
0
0
1
0
0
0
0
1
0
1
0
1
1
1
0
1
1
0
0
0
1
0
0
0
0
0
0
1
0
0
0
0
0
0
1
1
1
0
0
1
0
0
1
1
0
0
0
0
0
0
1
0
0
0
0
1
0
1
0
1
0
1
1
0
1
0
1
0
1
1
0
1
0
0
1
1
1
1
1
0
1
0
0
0
0
0
1
0
1
1
0
0
0
1
1
0
0
0
0
1
0
0
0
0
1
0
0
0
1
1
1
0
0
1
0
0
0
1
1
0
0
0
0
0
1
0
1
0
0
0
0
1
0
1
0
0
1
1
0
0
0
0
0
0
0
0
0
1
0
0
0
0
1
0
1
1
1
1
1
1
-1
1
0
1
1
1
1
0
1
0
0
1
1
1
1
1
0
1
0
0
0
0
0
1
0
1
1
0
0
0
1
1
0
0
0
0
1
0
0
0
0
1
0
1
0
0
1
0
0
1
0
1
0
0
1
1
0
0
0
0
0
1
0
1
0
0
0
0
0
1
1
0
0
1
1
0
1
1
0
0
0
0
0
0
0
0
0
0
0
1
0
1
1
0
0
1
0
1
1
-1
1
1
1
1
0
1
0
0
1
1
1
0
1
0
1
1
0
0
0
0
1
0
0
1
0
0
0
0
1
0
0
0
0
1
0
0
0
0
1
0
1
0
0
1
1
0
0
0
1
0
0
1
1
0
0
0
0
0
1
1
0
0
0
0
0
1
0
1
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
-1
0
0
1
1
1
-1
1
-1
0
0
0
1
0
-1
0
0
0
1
1
1
0
0
1
1
0
0
0
0
1
0
1
1
0
0
1
1
1
0
0
1
0
1
0
0
0
0
1
0
1
0
1
1
0
0
1
1
1
0
0
1
0
0
0
1
0
0
1
0
0
0
1
0
1
1

In [18]:
top = []

for v in g.vertices():
    if g.vp['bipartite'][v] == 0:
        top.append(v)

m = draw_gt.bipartite_layout(g, top)
move_gt(g, m)
save_gt(g, 'exemplo.net.gz')
r = netpixi.render('exemplo.net.gz')